---

## Set Up PostgreSQL (on a Mac)

Install with Homebrew:

```
$ brew update && brew install postgres
```

Or, if already installed,

```
$ brew update && brew upgrade postgres
```

Run `initdb` just once, basically to create the directory structure and such on disk that's needed for creating new databases. Note: The specified path should match the version of Postgres just installed!

```
$ initdb /usr/local/var/postgres9.5 -E utf8
```

To _manually_ start and stop a local Postgres server from running, use

```
$ pg_ctl -D /usr/local/var/postgres -l /usr/local/var/postgres/server.log start
$ pg_ctl -D /usr/local/var/postgres stop -s -m fast
```
        
Or to _automatically_ start a Postgres server (now and) at launch:

```
$ mkdir -p ~/Library/LaunchAgents
$ ln -sfv /usr/local/opt/postgresql/*.plist ~/Library/LaunchAgents
$ launchctl load ~/Library/LaunchAgents/homebrew.mxcl.postgresql.plist
```

Open the system paths file, `/etc/paths`, in a text editor, and move the line `/usr/local/bin` from the bottom of the file to the top (if it wasn't like this already). If you had to make a change to the file, reboot the computer. After rebooting, the command `which psql` command should return `/usr/local/bin/psql`.

Homebrew automatically created a database superuser account with the same login as your current Mac OS account. Let's create a dedicated user named `app` for connecting to and owning the app's database:

```
$ createuser --echo --pwprompt --superuser --createdb app
```

You'll be prompted to create a password — be sure to remember it or save it somewhere!

Homebrew also automatically created a database named `postgres` that may be used to log info for administrative tasks such as creating a user. Let's create a new database for this project:

```
$ createdb --echo --encoding=utf8 --host=<HOST> --port=<PORT> --username=app --owner=app <DBNAME>
```

We're using our `app` user to create the database, and assign it as the db's owner. **TODO:** Choose a `<DBNAME>` for this project!

To access the database through an interactive shell:

```
$ psql --host=<HOST> --port=<PORT> --username=app --dbname=<DBNAME>
```

While we're in the interactive shell, let's create the `pgcrypto` extension for secure password storage:

```
=# CREATE EXTENSION "pgcrypto";
```

Lastly, after exiting the postgres shell, define a `DATABASE_URL` environment variable containing this same information so that programs such as those in `cipy` can access it:

```
$ export DATABASE_URL=postgres://app<PASSWORD>@<HOST>/<DBNAME>
```

---

## Create Tables and Indexes from DDLs

In [1]:
import logging

import cipy

In [2]:
logger = logging.getLogger('cipy')
logger.setLevel(logging.DEBUG)

In [7]:
conn_creds = cipy.db.get_conn_creds(env_var='DATABASE_URL')

for ddl_name in cipy.DEFAULT_DDL_PATHS:
    ddl = cipy.db.get_ddl(ddl_name)
    print('\n{}'.format(ddl))
    
    pgdb = cipy.db.PostgresDB(conn_creds, ddl=ddl)
    pgdb.create_table(act=True)
    print('table created:\n\t{}'.format(ddl.create_table_statement()))
    
    pgdb.create_indexes(act=True)
    for index_statement in ddl.create_index_statements():
        print('index created:\n\t{}'.format(index_statement))


DDL(path='/Users/burtondewilde/Desktop/datakind/ci/conservation-intl/cipy/ddls/duplicates.yaml')
table created:
	CREATE TABLE IF NOT EXISTS duplicates (citation_id BIGINT, project_id INTEGER, canonical_citation_id INTEGER, duplicate_score REAL, is_confirmed BOOLEAN, confirming_user_id INTEGER) 
index created:
	CREATE INDEX IF NOT EXISTS duplicates_project_id_idx ON duplicates USING btree (project_id)

DDL(path='/Users/burtondewilde/Desktop/datakind/ci/conservation-intl/cipy/ddls/citations.yaml')
table created:
	CREATE TABLE IF NOT EXISTS citations (citation_id BIGSERIAL, project_id INTEGER, user_id INTEGER, inserted_ts TIMESTAMP(0) WITHOUT TIME ZONE, type_of_work VARCHAR(25), title VARCHAR(250), secondary_title VARCHAR(250), publication_year SMALLINT, publication_month SMALLINT, authors VARCHAR(100) ARRAY, abstract TEXT, keywords VARCHAR(100) ARRAY, type_of_reference VARCHAR(50), journal_column_name VARCHAR(100), volume VARCHAR(20), issue_number VARCHAR(20), doi VARCHAR(100), issn VAR